<a href="https://colab.research.google.com/github/gskumlehn/french-teacher/blob/main/wikipedia_search_audio_fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -y update && apt-get -y install ffmpeg portaudio19-dev
!pip install -q gTTS wikipedia googletrans==4.0.0-rc1 SpeechRecognition pydub

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,923 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [32.9 kB]
Get:13 https://ppa.la

In [2]:
!git clone https://github.com/gskumlehn/french-teacher.git

Cloning into 'french-teacher'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 362.84 KiB | 1.92 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
import os
import wikipedia
from gtts import gTTS
from googletrans import Translator
import speech_recognition as sr
from pydub import AudioSegment
from IPython.display import Audio, display

In [4]:
def speak_fr(text, filename="response_fr.mp3"):
    if os.path.exists(filename):
        os.remove(filename)
    tts = gTTS(text=text, lang='fr')
    tts.save(filename)
    return filename

def transcribe_en(audio_path, language='en-US'):
    r = sr.Recognizer()
    ext = os.path.splitext(audio_path)[1].lower()
    wav_path = audio_path
    if ext not in ['.wav', '.aiff', '.aif']:
        wav_path = os.path.splitext(audio_path)[0] + '.wav'
        AudioSegment.from_file(audio_path).export(wav_path, format='wav')
    with sr.AudioFile(wav_path) as source:
        audio = r.record(source)
    return r.recognize_google(audio, language=language)

In [5]:
wikipedia.set_lang("en")
translator = Translator()

audio_path = '/content/french-teacher/strawberry.wav'

try:
    query_text = transcribe_en(audio_path, language='en-US')
    print(f"You said: {query_text}")
except Exception as e:
    print(f"Error: {e}")
    query_text = ""

if query_text:
    keyword = query_text.strip().split()[0]
    try:
        summary_en = wikipedia.summary(keyword, sentences=3, auto_suggest=True, redirect=True)
        summary_fr = translator.translate(summary_en, src='en', dest='fr').text
        print("French summary:")
        print(summary_fr)
        mp3_path = speak_fr(summary_fr)
        display(Audio(mp3_path))
    except Exception as e:
        print(f"Error: {e}")

You said: strawberry
French summary:
La fraise du jardin (ou simplement la fraise; fragaria × ananassa) est une plante hybride largement cultivée cultivée dans le monde entier pour ses fruits.Le genre Fragaria, les fraises, se trouve dans la famille Rose, Rosaceae.Le fruit est apprécié pour son arôme, sa couleur rouge vif, sa texture juteuse et sa douceur.
